# EDA con Enfoque en CAC y LTV

Este cuaderno se centra en **preguntas de negocio** que generan valor al entender la relación entre:
- **CAC**: Costo de Adquisición de Cliente.
- **LTV**: Valor de Vida del Cliente.

Adicionalmente, explora aspectos clave como **suscripción**, **descuentos**, **categoría** del producto, **frecuencia de compras**, **método de pago**, etc. Con estas preguntas podrás tomar **decisiones estratégicas** sobre tu modelo de negocio, campañas de marketing y retención de clientes.

## Contenido:
1. **Importar librerías y cargar datos**
2. **Inspección inicial** (dimensiones, tipos de datos, nulos)
3. **Limpieza básica** (si es necesaria)
4. **Creación de columna** ratio `LTV/CAC` (rentabilidad por cliente)
5. **Preguntas de negocio** (groupby, filtros, etc.)
6. **Conclusiones**

In [1]:
!wget !wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv

--2025-03-13 23:56:35--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2025-03-13 23:56:35--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv [following]
--2025-03-13 23:56:35--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HT

## 1. Importar librerías y cargar datos
Asegúrate de tener tu archivo CSV con las columnas mencionadas: `CAC`, `LTV`, `Purchase Amount (USD)`, etc. Ajusta la ruta según tu archivo real.

In [2]:
import pandas as pd
import numpy as np


# Cargar el dataset
df = pd.read_csv("shopping_trends_LTV_CAC.csv")


# Mostramos las primeras filas para verificar
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,...,Next Day Air,Yes,Yes,49,PayPal,Weekly,NaN,1914,26.957746,26.957746
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574


## 2. Inspección inicial
1. **Dimensiones (filas x columnas)**
2. **Info general**: tipos de dato, nulos, etc.
3. **Valores nulos** por columna.

Si existen valores nulos en columnas importantes (por ejemplo, `CAC`, `LTV`, `Customer ID`), evalúa si los eliminarás o imputarás.

In [3]:
# Tabla de datos
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               3900 non-null   int64  
 1   Age                       3900 non-null   int64  
 2   Gender                    3900 non-null   object 
 3   Item Purchased            3900 non-null   object 
 4   Category                  3900 non-null   object 
 5   Purchase Amount (USD)     3900 non-null   int64  
 6   Location                  3900 non-null   object 
 7   Size                      3900 non-null   object 
 8   Color                     3900 non-null   object 
 9   Season                    3900 non-null   object 
 10  Review Rating             3900 non-null   float64
 11  Subscription Status       3900 non-null   object 
 12  Payment Method            3900 non-null   object 
 13  Shipping Type             3900 non-null   object 
 14  Discount

In [5]:
#Tabla de datos
data = {
    "Customer_ID": [1, 2, 3, 4,5],
    "Category": ["Clothing","Clothing","Clothing","footware","Clothing"],
    "CAC": [112.0, 95.0, 32.0, None, 47.0],
    "LTV": [1512, 1497, 1137, 1914, 1300]
}

# Creación del DataFrame
df = pd.DataFrame(data)

# 1. Inspección inicial: Dimensiones
print(f"Dimensiones del DataFrame: {df.shape[0]} filas, {df.shape[1]} columnas")

# 2. Información general sobre tipos de datos y valores nulos
print("\nInformación general:")
print(df.info())

# 3. Valores nulos por columna
print("\nValores nulos por columna:")
print(df.isnull().sum())

# 4. Evaluar columnas importantes (CAC, LTV, Customer_ID) y manejo de valores nulos
columnas_importantes = ['CAC', 'LTV', 'Customer_ID','Category']
for columna in columnas_importantes:
    nulos = df[columna].isnull().sum()
    if nulos > 0:
        print(f"\nLa columna '{columna}' tiene {nulos} valores nulos.")
        # Evaluación: imputar o eliminar según el caso
        if columna == "Customer_ID":
            print(f"**Sugerencia:** Eliminar filas con valores nulos en '{columna}', ya que es un identificador único.")
            df = df.dropna(subset=[columna])

# Mostrar el DataFrame después del manejo de nulos
print("\nDataFrame después del manejo de valores nulos:")
print(df)


Dimensiones del DataFrame: 5 filas, 4 columnas

Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Customer_ID  5 non-null      int64  
 1   Category     5 non-null      object 
 2   CAC          4 non-null      float64
 3   LTV          5 non-null      int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 292.0+ bytes
None

Valores nulos por columna:
Customer_ID    0
Category       0
CAC            1
LTV            0
dtype: int64

La columna 'CAC' tiene 1 valores nulos.

DataFrame después del manejo de valores nulos:
   Customer_ID  Category    CAC   LTV
0            1  Clothing  112.0  1512
1            2  Clothing   95.0  1497
2            3  Clothing   32.0  1137
3            4  footware    NaN  1914
4            5  Clothing   47.0  1300


## 3. Limpieza básica
Por ejemplo, si detectas `CAC <= 0` o `LTV <= 0` y consideras que no tienen sentido en tu negocio, podrías filtrar esos registros. Ajusta según tu criterio.

In [6]:
# Filtrar registros donde CAC > 0 y LTV > 0
df_limpio = df[(df['CAC'] > 0) & (df['LTV'] > 0)]

print("Registros originales:")
print(df)

print("\nRegistros después de la limpieza:")
print(df_limpio)

Registros originales:
   Customer_ID  Category    CAC   LTV
0            1  Clothing  112.0  1512
1            2  Clothing   95.0  1497
2            3  Clothing   32.0  1137
3            4  footware    NaN  1914
4            5  Clothing   47.0  1300

Registros después de la limpieza:
   Customer_ID  Category    CAC   LTV
0            1  Clothing  112.0  1512
1            2  Clothing   95.0  1497
2            3  Clothing   32.0  1137
4            5  Clothing   47.0  1300


## 4. Creación de columna: Ratio LTV/CAC
Es una métrica clave de la **rentabilidad** del cliente. Un valor > 1 indica que, en teoría, el cliente retorna más de lo que cuesta adquirirlo.

In [7]:
# Creación de la columna Ratio LTV/CAC
df_limpio['Ratio_LTV_CAC'] = df_limpio['LTV'] / df_limpio['CAC']

# Verificar el resultado
print("DataFrame con la nueva columna 'Ratio LTV/CAC':")
print(df_limpio)

DataFrame con la nueva columna 'Ratio LTV/CAC':
   Customer_ID  Category    CAC   LTV  Ratio_LTV_CAC
0            1  Clothing  112.0  1512      13.500000
1            2  Clothing   95.0  1497      15.757895
2            3  Clothing   32.0  1137      35.531250
4            5  Clothing   47.0  1300      27.659574


<ipython-input-7-11f34050aa0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['Ratio_LTV_CAC'] = df_limpio['LTV'] / df_limpio['CAC']


## 5. Preguntas de Negocio
En vez de usar gráficas o correlaciones, nos centramos en **preguntas que generen valor** al análisis. Cada una se resuelve con **groupby, filtros o estadísticas** simples.

1. **¿Cuántos clientes únicos hay y cuál es la edad promedio?**
2. **¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?**
3. **¿Cuál es la ratio LTV/CAC promedio?**
4. **¿Qué categoría tiene el mayor LTV en promedio?**
5. **¿En cuál categoría se justifica más el CAC?** (categoría con mayor ratio LTV/CAC)
6. **¿Los clientes suscritos tienen LTV mayor que los no suscritos?**
7. **¿Cuál es el método de pago más frecuente y su LTV promedio?**
8. **¿Quiénes usan descuentos con mayor frecuencia?** (Por ejemplo, ¿cierto género, cierto rango de edad?)
9. **¿Cuántos clientes aprovechan códigos promocionales (Promo Code Used)?** y su ratio LTV/CAC
10. **¿Cuál es la frecuencia de compra más común (Frequency of Purchases)?** y su LTV asociado.
11. **¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?**
12. **Entre los clientes con `Previous Purchases` > X, cuál es el gasto promedio y LTV promedio?**


### 5.1 ¿Cuántos clientes únicos hay y cuál es la edad promedio?

In [8]:
import pandas as pd

# Ejemplo de DataFrame
data = {
    "Customer_ID": [1, 2, 3, 4, 5],
    "Age": [55, 19, 50, 21, 45],
    "Purchase_Amount_USD": [53, 64, 73, 90, 49],
    "Previous_purchases" : [14, 2, 23, 49, 31],
}

df = pd.DataFrame(data)

# Contar clientes únicos
clientes_unicos = df['Customer_ID'].nunique()

# Calcular la edad promedio
edad_promedio = df['Age'].mean()

print(f"Número de clientes únicos: {clientes_unicos}")
print(f"Edad promedio: {edad_promedio:.2f} años")

Número de clientes únicos: 5
Edad promedio: 38.00 años


### 5.2 ¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?

In [9]:
# CAC promedio y el LTV promedio en todo el dataset
cac_promedio = df['Purchase_Amount_USD'].mean()
ltv_promedio = df['Purchase_Amount_USD'].sum() / clientes_unicos

print(f"CAC promedio: ${cac_promedio:.2f}")


CAC promedio: $65.80


### 5.3 ¿Cuál es la ratio LTV/CAC promedio?

In [10]:
# ratio LTV/CAC promedio
ratio_promedio = ltv_promedio / cac_promedio

print(f"Ratio LTV/CAC promedio: {ratio_promedio:.2f}")


Ratio LTV/CAC promedio: 1.00


### 5.4 ¿Qué categoría tiene el mayor LTV en promedio?

In [11]:
# tabla de datos (ajusta los valores.
data = {
    "Category": ["Clothing", "Clothing", "Clothing", "Footwear", "Clothing"],
    "CAC": [112.0, 95.0, 32.0, None, 47.0],
    "LTV": [1512, 1497, 1137, 1914, 1300],
}

# Creación del DataFrame
df = pd.DataFrame(data)

# Agrupar por categoría y calcular el promedio de LTV
ltv_promedio_por_categoria = df.groupby('Category')['LTV'].mean()

# Identificar la categoría con el mayor LTV promedio
categoria_mayor_ltv = ltv_promedio_por_categoria.idxmax()
ltv_mayor_promedio = ltv_promedio_por_categoria.max()

print("Categoría con el mayor LTV promedio:", categoria_mayor_ltv)
print(f"LTV promedio más alto: {ltv_mayor_promedio:.2f}")


Categoría con el mayor LTV promedio: Footwear
LTV promedio más alto: 1914.00


### 5.5 ¿En cuál categoría se justifica más el CAC? (mayor ratio LTV/CAC)

In [13]:
# categoría se justifica más el CAC? (mayor ratio LTV/CAC)
ratio_ltv_cac = ltv_promedio_por_categoria / cac_promedio

# Identificar la categoría con el mayor ratio LTV/CAC
categoria_mayor_ratio = ratio_ltv_cac.idxmax()
ratio_maximo = ratio_ltv_cac.max()

print("Categoría que justifica más el CAC:", categoria_mayor_ratio)
print(f"Ratio LTV/CAC más alto: {ratio_maximo:.2f}")



Categoría que justifica más el CAC: Footwear
Ratio LTV/CAC más alto: 29.09


### 5.6 ¿Los clientes suscritos tienen LTV mayor que los no suscritos?

In [16]:
# Los clientes suscritos tienen LTV mayor que los no suscritos
suscritos = df[df['Subscription Status'] == 'Yes']
no_suscritos = df[df['Subscription Status'] == 'No']

ltv_suscritos = suscritos['LTV'].mean()
ltv_no_suscritos = no_suscritos['LTV'].mean()

print(f"LTV promedio de clientes suscritos: {ltv_suscritos:.2f}")
print(f"LTV promedio de clientes no suscritos: {ltv_no_suscritos:.2f}")


KeyError: 'Subscription Status'

### 5.7 ¿Cuál es el método de pago más frecuente y su LTV promedio?

In [18]:
# método de pago más frecuente y su LTV promedio
metodo_pago_frecuencia = df['Payment Method'].value_counts()
metodo_pago_frecuencia_mas_frecuente = metodo_pago_frecuencia.idxmax()
ltv_promedio_por_metodo = df.groupby('Payment Method')['LTV'].mean()
ltv_promedio_mas_alto = ltv_promedio_por_metodo.max()

print("Método de pago más frecuente:", metodo_pago_frecuencia_mas_frecuente)
print(f"LTV promedio más alto: {ltv_promedio_mas_alto:.2f}")



KeyError: 'Payment Method'

### 5.8 ¿Quiénes usan descuentos con mayor frecuencia?
Podemos ver si hay algún grupo (por género, rango de edad, etc.) que destaca. A modo de ejemplo, segmentaremos por género:

In [19]:
# Clientes que usan descuentos con mayor frecuencia de acuerdo al Age
df. age

KeyError: 'Discount Applied'

Podrías repetir el ejercicio segmentando por **Age** (definiendo rangos) o por **Location**.

### 5.9 ¿Cuántos clientes aprovechan códigos promocionales y cuál es su ratio LTV/CAC?
El uso de promos puede modificar la rentabilidad.

### 5.10 ¿Cuál es la frecuencia de compra más común (Frequency of Purchases) y su LTV?
Esto ayuda a ver si, por ejemplo, los clientes que compran semanalmente tienen un LTV más alto.

### 5.11 ¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?

### 5.12 Entre los clientes con `Previous Purchases` > X, ¿cuál es el gasto promedio y el LTV promedio?
Define un umbral X según tu contexto. Por ejemplo, X = 20:

## 6. Conclusiones

En este Notebook, hemos abordado un **EDA orientado a negocio** en el que:
1. **Identificamos** la base total de clientes y su edad promedio.
2. **Calculamos** el CAC y LTV promedio, y la ratio LTV/CAC global.
3. **Segregamos** por categorías, suscripción, uso de descuentos, métodos de pago, etc.
4. **Resolvimos** preguntas de negocio que ayudan a:
   - **Identificar** cuáles categorías o ubicaciones ofrecen mejor rentabilidad (ratio LTV/CAC).
   - **Entender** si la suscripción eleva el LTV.
   - **Explorar** la efectividad de promos y descuentos.
   - **Diferenciar** frecuencias de compra y ver su impacto en LTV.
   - **Focalizar** inversiones de marketing en los segmentos más prometedores.

Este tipo de **análisis** proporciona las bases para **estrategias** de retención, adquisición y segmentación de clientes. ¡Espero que te sea útil para **generar valor** en tu negocio!